In [ ]:
!pip install soynlp

In [ ]:
!git clone https://github.com/ssut/py-hanspell

In [ ]:
import os
os.chdir("./py-hanspell")
!python setup.py install

In [ ]:
os.chdir("../")

In [ ]:
import json
import re
from tqdm.notebook import tqdm
from copy import deepcopy
import pandas as pd
from hanspell import spell_checker

In [ ]:
def jsonload(fname, encoding="utf-8"):
    with open(fname, encoding=encoding) as f:
        j = json.load(f)

    return j


# json 개체를 파일이름으로 깔끔하게 저장
def jsondump(j, fname):
    with open(fname, "w", encoding="UTF8") as f:
        json.dump(j, f, ensure_ascii=False)

        
# jsonl 파일 읽어서 list에 저장
def jsonlload(fname, encoding="utf-8"):
    json_list = []
    with open(fname, encoding=encoding) as f:
        for line in f.readlines():
            json_list.append(json.loads(line))
    return json_list

In [ ]:
train_file_path = "./Original_data/nikluge-sa-2022-train.jsonl"
dev_file_path = "./Original_data/nikluge-sa-2022-dev.jsonl"

train_data = jsonlload(train_file_path)
dev_data = jsonlload(dev_file_path)

In [ ]:
train_data.extend(dev_data)

In [ ]:
train_data

In [ ]:
entity_property_id_to_name = [
    '제품 전체#일반', '제품 전체#가격', '제품 전체#디자인', '제품 전체#품질', '제품 전체#편의성', '제품 전체#인지도', '제품 전체#다양성',
    '본품#일반', '본품#디자인', '본품#품질', '본품#편의성', '본품#다양성', '본품#가격', '본품#인지도',
    '패키지/구성품#일반', '패키지/구성품#가격', '패키지/구성품#디자인', '패키지/구성품#품질', '패키지/구성품#편의성', '패키지/구성품#다양성',
    '브랜드#일반', '브랜드#가격', '브랜드#디자인', '브랜드#품질', '브랜드#인지도',
                    ]

entity_property_count = {entity_property_id_to_name[i]: 0 for i in range(len(entity_property_id_to_name))}

In [ ]:
entity_property_count

In [ ]:
for pair in train_data:
    for detail in pair["annotation"]:
        for entity_property in entity_property_id_to_name:
            if entity_property == detail[0]:
                entity_property_count[entity_property] += 1

In [ ]:
entity_property_count

In [ ]:
double_index = []

for i,pair in enumerate(train_data):
    if len(pair["annotation"]) > 1:
        double_index.append(i)

In [ ]:
single_index = []

for i,pair in enumerate(train_data):
    if len(pair["annotation"]) == 1:
        single_index.append(i)

In [ ]:
print(len(double_index))

In [ ]:
print(len(single_index))

In [ ]:
print(len(train_data))

In [ ]:
single_index_train_data = deepcopy(train_data)
double_index.sort(reverse=True)
for i in double_index:
    del single_index_train_data[i]

In [ ]:
double_index_train_data = deepcopy(train_data)
single_index.sort(reverse=True)
for i in single_index:
    del double_index_train_data[i]

In [ ]:
print(len(single_index_train_data))

In [ ]:
print(len(double_index_train_data))

In [ ]:
print(len(train_data))

In [ ]:
entity_property_id_to_name = [
    '제품 전체#일반', '제품 전체#가격', '제품 전체#디자인', '제품 전체#품질', '제품 전체#편의성', '제품 전체#인지도', '제품 전체#다양성',
    '본품#일반', '본품#디자인', '본품#품질', '본품#편의성', '본품#다양성', '본품#가격', '본품#인지도',
    '패키지/구성품#일반', '패키지/구성품#가격', '패키지/구성품#디자인', '패키지/구성품#품질', '패키지/구성품#편의성', '패키지/구성품#다양성',
    '브랜드#일반', '브랜드#가격', '브랜드#디자인', '브랜드#품질', '브랜드#인지도',
                    ]

entity_property_single_count = {entity_property_id_to_name[i]: 0 for i in range(len(entity_property_id_to_name))}
entity_property_double_count = {entity_property_id_to_name[i]: 0 for i in range(len(entity_property_id_to_name))}

In [ ]:
for pair in single_index_train_data:
    for detail in pair["annotation"]:
        for entity_property in entity_property_id_to_name:
            if entity_property == detail[0]:
                entity_property_single_count[entity_property] += 1
entity_property_single_count

In [ ]:
for pair in double_index_train_data:
    for detail in pair["annotation"]:
        for entity_property in entity_property_id_to_name:
            if entity_property == detail[0]:
                entity_property_double_count[entity_property] += 1
entity_property_double_count

In [ ]:
entity_property_count = {entity_property_id_to_name[i]: 0 for i in range(len(entity_property_id_to_name))}
for pair in train_data:
    for detail in pair["annotation"]:
        for entity_property in entity_property_id_to_name:
            if entity_property == detail[0]:
                entity_property_count[entity_property] += 1
entity_property_count

In [ ]:
df = pd.DataFrame(columns = ["id", "sentence_form", "entity", "polarity"])

In [ ]:
for i,pair in enumerate(single_index_train_data):
    df.at[i, "id"] = pair["id"]
    df.at[i, "sentence_form"] = pair["sentence_form"]
    df.at[i, "entity"] = pair["annotation"][0][0]
    df.at[i, "polarity"] = pair["annotation"][0][2]

In [ ]:
df

In [ ]:
A = df[df['entity'] == '본품#인지도'].index
B = df[df['entity'] == '패키지/구성품#가격'].index

In [ ]:
temp1 = df.iloc[A]
temp2 = df.iloc[B]

In [ ]:
df.drop(A, inplace=True)
df.drop(B, inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split

target = df["entity"]
df_train, df_val = train_test_split(df, test_size = 0.1, shuffle=True, stratify=target, random_state=34)

In [ ]:
df_train = pd.concat([df_train, temp1])
df_train = pd.concat([df_train, temp2])
df_train.reset_index(drop=True, inplace=True)
df_val.reset_index(drop=True, inplace=True)

In [ ]:
df_train

In [ ]:
df_val

In [ ]:
train = []
dev = []

for i in range(len(df_train)):
    train.append({"id": df_train.at[i, 'id'], "sentence_form": df_train.at[i, 'sentence_form'], "annotation": [[df_train.at[i, 'entity'], [], df_train.at[i, 'polarity']]]})
    
for i in range(len(df_val)):
    dev.append({"id": df_val.at[i, 'id'], "sentence_form": df_val.at[i, 'sentence_form'], "annotation": [[df_val.at[i, 'entity'], [], df_val.at[i, 'polarity']]]})

In [ ]:
train.extend(double_index_train_data)

In [ ]:
train

In [ ]:
dev

In [ ]:
print(len(train))

In [ ]:
print(len(dev))

In [ ]:
def jsonl_store(raw_data, fname, encoding="utf-8"):
    with open(f"{fname}", encoding=encoding, mode="w") as f:
        for raw in raw_data:
            f.write(json.dumps(raw, ensure_ascii=False) + "\n")

# Data Preprocess : Remove words and digits containing digits

In [ ]:
import re
from soynlp.normalizer import *

def clean_str(text):
    text = re.sub("(&name&|&affiliation&|&social-security-num&|&tel-num&|&card-num&|&bank-account&|&num&|&online-account&)[^\s]*", "", text)
    text = only_text(text)
    text = emoticon_normalize(text, num_repeats=3)
    text = repeat_normalize(text, num_repeats=2)
    text = spell_checker.check(text).checked
    text = only_hangle(text)
    
    
    return text.strip()

In [ ]:
print(len(train))
print(len(dev))

In [ ]:
for i in tqdm(range(len(train))):
    sent = train_data[i]["sentence_form"]
    train_data[i]["sentence_form"] = clean_str(sent)

In [ ]:
for i in tqdm(range(len(dev))):
    sent = dev_data[i]["sentence_form"]
    dev_data[i]["sentence_form"] = clean_str(sent)

In [ ]:
train_name = "./Preprocessed_data/acc/train_acc_9_1_preprocessed.jsonl"
dev_name = "./Preprocessed_data/acc/dev_acc_9_1_preprocessed.jsonl"

In [ ]:
jsonl_store(train, train_name)
jsonl_store(dev, dev_name)